In [1]:
import os
from PIL import Image
from torch.utils.data import Dataset
from torchvision import transforms


class SatelliteSegmentationDataset(Dataset):
    # ну тут просто клас для датасету зі знімками і масками
    def __init__(self, image_dir, mask_dir, transform_img=None, transform_mask=None):
        self.image_dir = image_dir
        self.mask_dir = mask_dir
        self.transform_img = transform_img
        self.transform_mask = transform_mask
        self.image_filenames = sorted([
            f for f in os.listdir(self.image_dir) if f.endswith(".png")
        ])

    def __len__(self):
        return len(self.image_filenames)

    def __getitem__(self, idx):
        # тут беремо відповідну картинку і маску
        img_name = self.image_filenames[idx]
        img_path = os.path.join(self.image_dir, img_name)
        mask_path = os.path.join(self.mask_dir, img_name)

        # відкриваємо і те, і те
        image = Image.open(img_path).convert("RGB")
        mask = Image.open(mask_path)


        if self.transform_img:
            image = self.transform_img(image)
        if self.transform_mask:
            mask = self.transform_mask(mask)

        return image, mask  # повертаємо як є


In [2]:
from torch.utils.data import DataLoader
from torchvision.transforms import ToTensor
import torchvision.transforms as T

# Шляхи
image_dir = "segmentation/train/images"
mask_dir = "segmentation/train/masks"

# Трансформації
transform_img = ToTensor()
transform_mask = T.Compose([
    T.PILToTensor(),          # (C, H, W)
    T.Lambda(lambda x: x[0:1, :, :])  
])


# Ініціалізація
dataset = SatelliteSegmentationDataset(
    image_dir=image_dir,
    mask_dir=mask_dir,
    transform_img=transform_img,
    transform_mask=transform_mask
)

loader = DataLoader(dataset, batch_size=4, shuffle=True)
images, masks = next(iter(loader))

print("images:", images.shape)
print("masks:", masks.shape)


images: torch.Size([4, 3, 640, 640])
masks: torch.Size([4, 1, 640, 640])
